In [1]:
# importing required libraries 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from datetime import date
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# installing the chrome driver & URL to be scrapped 
driver= webdriver.Chrome(r"C:\Users\arkay\Downloads\chromedriver_win32 (1)\chromedriver.exe")
url='https://in.indeed.com/'
driver.get(url)
driver.maximize_window()
time.sleep(5)

In [3]:
# Using selenium automation to reach the desired page through code 
driver.find_element_by_xpath("//*[@id='text-input-what']").send_keys('Business Analyst')
driver.find_element_by_xpath("//*[@id='text-input-where']").send_keys('Gurgaon')
driver.find_element_by_xpath("//*[@id='whatWhereFormId']/div[3]/button").click()

In [4]:
# Let's write the codes
driver.implicitly_wait(1)
titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]
Time=[]
Date=[]
days_posted=[]

In [5]:
# lets scrap first 6 pages
for i in range(0,6):
    
    job_posts= driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')
    
    for job in job_posts:
       
    #.  not all companies have review
        try:
            review = job.find_element_by_xpath('.//span[@class="ratingsContent"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            salary = job.find_element_by_xpath('.//span[@class="salaryText"]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            location = job.find_element_by_xpath('.//span[contains(@class,"location")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            title  = job.find_element_by_xpath('.//h2[@class="title"]//a').text
        except:
            title = job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="title")
        titles.append(title)
        links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))
        companies.append(job.find_element_by_xpath('.//span[@class="company"]').text)
        
        today=date.today()
        Date.append(today.strftime("%d/%m/%Y"))
        now=datetime.now()
        Time.append(now.strftime("%H:%M:%S"))
        
        try:
            dp=job.find_element_by_class_name("date ").text
        except:
            dp='None'
            days_posted.append(dp)
    
    try:
        next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
        next_page.click()

    except:
        next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@aria-label="Next"]//span[@class="np"]"}
  (Session info: chrome=87.0.4280.88)


In [6]:
descriptions=[]
for link in links:
    
    driver.get(link)
    try:
        jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    except:
        jd='None'
    descriptions.append(jd)

In [7]:
df_Indeed=pd.DataFrame()
df_Indeed['Title']=titles
df_Indeed['Company']=companies
df_Indeed['Location']=locations
df_Indeed['Link']=links
df_Indeed['Review']=reviews
df_Indeed['Salary']=salaries
df_Indeed['Description']=descriptions
df_Indeed['Crawl Time']=Time
df_Indeed['Crawl Date']=Date

In [8]:
df_Indeed.head()

,Title,Company,Location,Link,Review,Salary,Description,Crawl Time,Crawl Date
0,Business Analyst,Audacia Ventures,"New Delhi, Delhi",https://in.indeed.com/company/Audacia-Ventures...,None,"₹35,000 - ₹45,000 a month",About Audacia Ventures\n*\nAudacia Ventures is...,20:17:33,04/01/2021
1,Business Analyst,Caliber Hunt,"Delhi, Delhi",https://in.indeed.com/rc/clk?jk=9ed1b43619cdfe...,None,"₹10,00,000 a year",Able to make Business Requirement Document (BR...,20:17:34,04/01/2021
2,Data Analyst,Shriram Automall India Ltd,"Delhi, Delhi",https://in.indeed.com/company/Shriram-Automall...,3.6,"₹50,000 a month",Data Analyst:\nJob Description\n· Interpret da...,20:17:35,04/01/2021
3,Business Analyst,KBR,"Delhi, Delhi",https://in.indeed.com/rc/clk?jk=4b8cf6733c621a...,4.1,None,Title:\nBusiness Analyst\nPOSITION DESCRIPTION...,20:17:36,04/01/2021
4,Business Analyst,MedTourEasy,"New Delhi, Delhi",https://in.indeed.com/rc/clk?jk=1f37fe2b290749...,None,"₹7,00,000 a year",We are looking for a Business Analyst who will...,20:17:37,04/01/2021


In [9]:
df_Indeed.to_csv('df_Indeedgurgaon.csv')